In [1]:
from datetime import datetime

from pymongo import MongoClient
import pandas as pd

import constants
from tqdm import tqdm


In [2]:
mongo = MongoClient(constants.mongo_conn_string)
mongo_lt_col = mongo[constants.mongo_db][constants.mongo_lt_annotated_col]

In [3]:
query = {}

cursor = mongo_lt_col.find(query, {"_id":0, "text":0})

data = [i for i in cursor]
print(len(data))

df = pd.DataFrame(data)
print(df.shape)
df.head()

58829
(58829, 10)


,para_id,source,cabinet_ents,date,year,month,day,cabinet,cabinet_no,president
0,614bbf4a4307555f8af6dd8f,https://www.delfi.lt/news/daily/lithuania/zemd...,[Žemės Ūkio],2000-01-27 13:20:46,2000,1,27,,,Valdas Adamkus
1,614bf1fc4307555f8af74454,https://www.delfi.lt/news/daily/lithuania/liet...,[Prezidentas],2000-02-01 21:30:03,2000,2,1,,,Valdas Adamkus
2,614b53a14307555f8af628a1,https://www.delfi.lt/news/daily/lithuania/lega...,[Švietimo ir mokslo],2000-02-02 15:55:13,2000,2,2,,,Valdas Adamkus
3,614bdfcb4307555f8af720cd,https://www.delfi.lt/news/daily/lithuania/rola...,[Prezidentas],2000-02-07 16:00:50,2000,2,7,,,Valdas Adamkus
4,614ba5d54307555f8af6a8cd,https://www.delfi.lt/news/daily/lithuania/libe...,[Prezidentas],2000-02-07 21:23:02,2000,2,7,,,Valdas Adamkus


In [4]:
#df.to_parquet("Data.parquet")

# Basic Data

In [7]:
df.year.value_counts()

2021    5959
2020    5858
2019    4702
2018    3822
2017    2999
2016    2944
2009    2755
2015    2735
2011    2575
2012    2503
2014    2336
2010    2334
2013    2165
2003    2111
2002    1989
2004    1935
2005    1805
2006    1708
2008    1586
2001    1505
2007    1484
2000     654
2022     365
Name: year, dtype: int64

In [8]:
df.cabinet.value_counts()

Saulius Skvernelis      17194
Andrius Kubilius        10264
Algirdas Butkevičius    10149
Algirdas Brazauskas      9302
Ingrida Šimonytė         6666
Gediminas Kirkilas       3653
Rolandas Paksas           932
                          669
Name: cabinet, dtype: int64

In [10]:
df.president.value_counts()

Dalia Grybauskaite    28200
Gitanas Nausėda       14397
Valdas Adamkus        13362
Rolandas Paksas        2586
                        284
Name: president, dtype: int64

# Adding dummies for media & presidential attention for each policy sphere

In [12]:
ministries = set()
for i in list(df.cabinet_ents):
    for a in i:
        ministries.add(a)
    
for item in tqdm(list(ministries)):
    df[item]= df.apply(lambda x : int(item in x["cabinet_ents"]), axis = 1)


def prez_attention( list_min , minister):
    if "Prezidentas" in list_min and minister in list_min:
        return 1
    else:
        return 0
    
for item in tqdm(list(ministries)):
    if item != "Prezidentas":
        df["president_"+item]= df.apply(lambda x : prez_attention(x["cabinet_ents"], item), axis = 1)
    
df.head() 

100%|██████████| 18/18 [00:12<00:00,  1.49it/s]


,para_id,source,cabinet_ents,date,year,month,day,cabinet,cabinet_no,president,...,president_Energetikos,president_Ekonomikos ir inovacijų,president_Teisingumo,president_Finansų,president_Užsienio reikalų,president_Socialinės apsaugos ir darbo,president_Sveikatos apsaugos,president_Švietimo ir mokslo,"president_Švietimo, mokslo ir sporto",president_Vidaus reikalų
0,614bbf4a4307555f8af6dd8f,https://www.delfi.lt/news/daily/lithuania/zemd...,[Žemės Ūkio],2000-01-27 13:20:46,2000,1,27,,,Valdas Adamkus,...,0,0,0,0,0,0,0,0,0,0
1,614bf1fc4307555f8af74454,https://www.delfi.lt/news/daily/lithuania/liet...,[Prezidentas],2000-02-01 21:30:03,2000,2,1,,,Valdas Adamkus,...,0,0,0,0,0,0,0,0,0,0
2,614b53a14307555f8af628a1,https://www.delfi.lt/news/daily/lithuania/lega...,[Švietimo ir mokslo],2000-02-02 15:55:13,2000,2,2,,,Valdas Adamkus,...,0,0,0,0,0,0,0,0,0,0
3,614bdfcb4307555f8af720cd,https://www.delfi.lt/news/daily/lithuania/rola...,[Prezidentas],2000-02-07 16:00:50,2000,2,7,,,Valdas Adamkus,...,0,0,0,0,0,0,0,0,0,0
4,614ba5d54307555f8af6a8cd,https://www.delfi.lt/news/daily/lithuania/libe...,[Prezidentas],2000-02-07 21:23:02,2000,2,7,,,Valdas Adamkus,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df.columns

Index(['para_id', 'source', 'cabinet_ents', 'date', 'year', 'month', 'day',
       'cabinet', 'cabinet_no', 'president', 'Krašto apsaugos', 'Premjeras',
       'Susisiekimo', 'Žemės Ūkio', 'Kultūros', 'Ūkio', 'Aplinkos ',
       'Energetikos', 'Ekonomikos ir inovacijų', 'Teisingumo', 'Finansų',
       'Užsienio reikalų', 'Prezidentas', 'Socialinės apsaugos ir darbo',
       'Sveikatos apsaugos', 'Švietimo ir mokslo',
       'Švietimo, mokslo ir sporto', 'Vidaus reikalų',
       'president_Krašto apsaugos', 'president_Premjeras',
       'president_Susisiekimo', 'president_Žemės Ūkio', 'president_Kultūros',
       'president_Ūkio', 'president_Aplinkos ', 'president_Energetikos',
       'president_Ekonomikos ir inovacijų', 'president_Teisingumo',
       'president_Finansų', 'president_Užsienio reikalų',
       'president_Socialinės apsaugos ir darbo',
       'president_Sveikatos apsaugos', 'president_Švietimo ir mokslo',
       'president_Švietimo, mokslo ir sporto', 'president_Vidaus rei

In [14]:
df.to_parquet("DataEnhanced.parquet")